In [1]:
import datetime

import drms
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.map import Map

In [2]:
start_time = datetime.datetime(2011, 2, 14, 0, 0, 0)
start_time

datetime.datetime(2011, 2, 14, 0, 0)

In [3]:
dt = datetime.timedelta(seconds=10)
dt

datetime.timedelta(seconds=10)

In [6]:
c = drms.Client(email="mgjeon@khu.ac.kr")

# SHARP

In [5]:
c.info("hmi.sharp_cea_720s").segments

,type,units,protocol,dims,note
name,,,,,
magnetogram,int,Gauss,fits,VARxVAR,Line-of-sight magnetogram in CEA projection
bitmap,char,Enumerated,fits,VARxVAR,Mask for the patch in CEA coordinates
Dopplergram,int,m/s,fits,VARxVAR,Dopplergram in CEA projection
continuum,int,DN/s,fits,VARxVAR,Intensitygram in CEA projection
Bp,int,Gauss,fits,VARxVAR,"B_phi, positive westward"
Bt,int,Gauss,fits,VARxVAR,"B_theta, positive southward"
Br,int,Gauss,fits,VARxVAR,"B_r, positive up"
Bp_err,int,Gauss,fits,VARxVAR,Standard deviation of B_phi
Bt_err,int,Gauss,fits,VARxVAR,Standard deviation of B_theta


In [6]:
c.pkeys("hmi.sharp_cea_720s")

['HARPNUM', 'T_REC']

In [7]:
find_ar = c.query("hmi.Mharp_720s[][%sZ]" % start_time,
                   key = ["NOAA_AR", "HARPNUM"])
find_ar

,NOAA_AR,HARPNUM
0,11157,364
1,11156,367
2,0,369
3,11159,371
4,0,372
5,11158,377
6,0,378
7,0,380
8,0,381
9,11160,384


In [8]:
hmi_results = Fido.search(
    a.Time(start_time - dt, start_time + dt),
    a.jsoc.Notify("mgjeon@khu.ac.kr"),
    a.jsoc.Series("hmi.sharp_cea_720s"),
    a.jsoc.PrimeKey('HARPNUM', '377'),
    a.jsoc.Segment("Bp") & a.jsoc.Segment("Bt") & a.jsoc.Segment("Br"),
)

In [9]:
hmi_results

T_REC,TELESCOP,INSTRUME,WAVELNTH,CAR_ROT
str23,str7,str9,float64,int64
2011.02.14_00:00:00_TAI,SDO/HMI,HMI_SIDE1,6173.0,2106


In [ ]:
hmi_files = Fido.fetch(hmi_results, path="download/11158_single/hmi/{file}")

# AIA

In [7]:
c.info("aia.lev1_euv_12s").segments

,type,units,protocol,dims,note
name,,,,,
image,int,None,link via lev1,None,AIA level 1 image
spikes,int,None,link via lev1,None,Cosmic ray information


In [8]:
c.pkeys("aia.lev1_euv_12s")

['T_REC', 'WAVELNTH']

In [9]:
wavelengths = [94, 
               131, 
               171, 
               193, 
               211, 
               304, 
               335]

In [10]:
aia_results = {}

for w in wavelengths:
    aia_results[str(w)] = Fido.search(
        a.Time(start_time - dt, start_time + dt),
        a.jsoc.Notify("mgjeon@khu.ac.kr"),
        a.jsoc.Series("aia.lev1_euv_12s"),
        a.jsoc.Segment("image"),
        a.Wavelength(w*u.angstrom))

In [11]:
for key, value in aia_results.items():
    print(key, value)

94 Results from 1 Provider:

2 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2011-02-13T23:59:50Z  SDO/AIA    AIA_4       94    2106
2011-02-14T00:00:02Z  SDO/AIA    AIA_4       94    2106


131 Results from 1 Provider:

2 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2011-02-13T23:59:50Z  SDO/AIA    AIA_1      131    2106
2011-02-14T00:00:02Z  SDO/AIA    AIA_1      131    2106


171 Results from 1 Provider:

2 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2011-02-13T23:59:50Z  SDO/AIA    AIA_3      171    2106
2011-02-14T00:00:02Z  SDO/AIA    AIA_3      171    2106


193 Results from 1 Provider:

2 Result

In [12]:
for wavelnth, result in aia_results.items():
    Fido.fetch(result, path=f"download/11158_single/aia/{wavelnth}/" + "{file}")

Export request pending. [id=JSOC_20240411_000717, status=2]
Waiting for 0 seconds...
2 URLs found for download. Full request totalling 14MB


Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]

aia.lev1_euv_12s.2011-02-13T235952Z.94.image_lev1.fits:   0%|          | 0.00/7.50M [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-02-14T000004Z.94.image_lev1.fits:   0%|          | 0.00/7.50M [00:00<?, ?B/s]

Export request pending. [id=JSOC_20240411_000719, status=2]
Waiting for 0 seconds...
2 URLs found for download. Full request totalling 17MB


...

aia.lev1_euv_12s.2011-02-14T000010Z.304.image_lev1.fits:   0%|          | 0.00/9.95M [00:00<?, ?B/s]

Export request pending. [id=JSOC_20240411_000742, status=2]
Waiting for 0 seconds...
2 URLs found for download. Full request totalling 15MB


Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]

aia.lev1_euv_12s.2011-02-13T235953Z.335.image_lev1.fits:   0%|          | 0.00/7.63M [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-02-14T000005Z.335.image_lev1.fits:   0%|          | 0.00/7.63M [00:00<?, ?B/s]